In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
np.random.seed(42)

## Week 10: Clustering Analysis in Python
Nov 7, 2022

## Housekeeping

- Assignment #5 (optional) due one week from today (11/14)
- Assignment #6 (optional) will be assigned this Wednesday 11/9, due on 11/21
- Remember, you have to do one of homeworks #4, #5, or #6

## Recap

- Last week: urban street networks + interactive web maps
- New tools: OSMnx, Pandana, and Folium

## Where we left off last week: choropleth maps in Folium

**Example:** A map of households without internet for US counties

Two ways:
- The easy way: `folium.Choropleth`
- The hard way: `folium.GeoJson`

In [2]:
import folium

Load the data, remove counties with no households and add our new column:

In [3]:
# Load CSV data from the data/ folder
census_data = pd.read_csv("./data/internet_avail_census.csv", dtype={"geoid": str})

# Remove counties with no households
valid = census_data['universe'] > 0
census_data = census_data.loc[valid]

# Calculate the percent without internet
census_data['percent_no_internet'] = census_data['no_internet'] / census_data['universe']

In [4]:
census_data.head()

,NAME,universe,no_internet,state,county,geoid,percent_no_internet
0,"Washington County, Mississippi",18299.0,6166.0,28,151,28151,0.336958
1,"Perry County, Mississippi",4563.0,1415.0,28,111,28111,0.310103
2,"Choctaw County, Mississippi",3164.0,1167.0,28,19,28019,0.368837
3,"Itawamba County, Mississippi",8706.0,1970.0,28,57,28057,0.226281
4,"Carroll County, Mississippi",3658.0,1218.0,28,15,28015,0.332969


Load the counties geometry data too:

In [5]:
# Load counties GeoJSOn from the data/ folder
counties = gpd.read_file("./data/us-counties-10m.geojson")

In [6]:
counties.head()

,id,geometry
0,53073,"MULTIPOLYGON (((-120.85361 49.00011, -120.7674..."
1,30105,"POLYGON ((-106.11238 48.99904, -106.15187 48.8..."
2,30029,"POLYGON ((-114.06985 48.99904, -114.05908 48.8..."
3,16021,"POLYGON ((-116.04755 49.00065, -116.04755 48.5..."
4,30071,"POLYGON ((-107.17840 49.00011, -107.20712 48.9..."


## The hard way: use folium.GeoJson

- **The good:**
    - More customizable, and can add user interaction
- **The bad:**
    - Requires more work
    - No way to add a legend, see [this open issue on GitHub](https://github.com/python-visualization/folium/issues/528)

## The steps involved

1. Join data and geometry features into a single GeoDataFrame
1. Define a function to style features based on data values
1. Create GeoJSON layer and add it to the map

### Step 1: Join the census data and features

**Note:** this is different than using `folium.Choropleth`, where data and features are stored in two separate data frames.

In [7]:
# Merge the county geometries with census data
# Left column: "id"
# Right column: "geoid"
census_joined = counties.merge(census_data, left_on="id", right_on="geoid")

In [8]:
census_joined.head()

,id,geometry,NAME,universe,no_internet,state,county,geoid,percent_no_internet
0,53073,"MULTIPOLYGON (((-120.85361 49.00011, -120.7674...","Whatcom County, Washington",85008.0,9189.0,53,73,53073,0.108096
1,30105,"POLYGON ((-106.11238 48.99904, -106.15187 48.8...","Valley County, Montana",3436.0,672.0,30,105,30105,0.195576
2,30029,"POLYGON ((-114.06985 48.99904, -114.05908 48.8...","Flathead County, Montana",38252.0,5662.0,30,29,30029,0.148018
3,16021,"POLYGON ((-116.04755 49.00065, -116.04755 48.5...","Boundary County, Idaho",4605.0,1004.0,16,21,16021,0.218024
4,30071,"POLYGON ((-107.17840 49.00011, -107.20712 48.9...","Phillips County, Montana",1770.0,484.0,30,71,30071,0.273446


### Step 2: Normalize the data column to 0 to 1

- We will use a matplotlib color map that requires data to be between 0 and 1
- Normalize our "percent_no_internet" column to be between 0 and 1

In [9]:
# Minimum
min_val = census_joined['percent_no_internet'].min()

# Maximum
max_val = census_joined['percent_no_internet'].max()

# Calculate a normalized column
normalized = (census_joined['percent_no_internet'] - min_val) / (max_val - min_val)

# Add to the dataframe
census_joined['percent_no_internet_normalized'] = normalized

### Step 3: Define our style functions

- Create a matplotlib colormap object using `plt.get_cmap()`
- Color map objects are *functions*: give the function a number between 0 and 1 and it will return a corresponding color from the color map
- Based on the feature data, evaluate the color map and convert to a hex string

In [10]:
import matplotlib.colors as mcolors

In [11]:
# Use a red-purple colorbrewer color scheme
cmap = plt.get_cmap('RdPu')

In [12]:
# The minimum value of the color map as an RGB tuple
cmap(0)

(1.0, 0.9686274509803922, 0.9529411764705882, 1.0)

In [13]:
# The minimum value of the color map as a hex string
mcolors.rgb2hex(cmap(0.0))

'#fff7f3'

In [14]:
# The maximum value of the color map as a hex string
mcolors.rgb2hex(cmap(1.0))

'#49006a'

In [15]:
def get_style(feature):
    """
    Given an input GeoJSON feature, return a style dict.
    
    Notes
    -----
    The color in the style dict is determined by the 
    "percent_no_internet_normalized" column in the 
    input "feature".
    """
    # Get the data value from the feature
    value = feature['properties']['percent_no_internet_normalized']
    
    # Evaluate the color map
    # NOTE: value must between 0 and 1
    rgb_color = cmap(value) # this is an RGB tuple
    
    # Convert to hex string
    color = mcolors.rgb2hex(rgb_color)
    
    # Return the style dictionary
    return {'weight': 0.5, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

In [16]:
def get_highlighted_style(feature):
    """
    Return a style dict to use when the user highlights a 
    feature with the mouse.
    """
    
    return {"weight": 3, "color": "black"}

### Step 4: Convert our data to GeoJSON

- **Tip**: To limit the amount of data Folium has to process, it's best to trim our GeoDataFrame to only the columns we'll need before converting to GeoJSON
- You can use the `.to_json()` function to convert to a GeoJSON string

In [17]:
needed_cols = ['NAME', 'percent_no_internet', 'percent_no_internet_normalized', 'geometry']
census_json = census_joined[needed_cols].to_json()

In [18]:
# STEP 1: Initialize the map
m = folium.Map(location=[40, -98], zoom_start=4)

# STEP 2: Add the GeoJson to the map
folium.GeoJson(
    census_json, # The geometry + data columns in GeoJSON format
    style_function=get_style, # The style function to color counties differently
    highlight_function=get_highlighted_style, 
    tooltip=folium.GeoJsonTooltip(['NAME', 'percent_no_internet'])
).add_to(m)



# avoid a rendering bug by saving as HTML and re-loading
m.save('percent_no_internet.html')

## And viola! 

The hard way is **harder**, but we have a tooltip and highlight interactivity!

In [19]:
from IPython.display import IFrame

In [20]:
IFrame('percent_no_internet.html', width=800, height=500)

## At-home exercise: Can we repeat this with altair?

Try to replicate the above interactive map exactly (minus the background tiles). This includes:

- Using the red-purple colorbrewer scheme
- Having a tooltip with the percentage and county name

**Note:** Altair's syntax is similar to the folium.Choropleth syntax — you should pass the counties GeoDataFrame to the `alt.Chart()` and then use the `transform_lookup()` to merge those geometries to the census data and pull in the census data column we need ("percent_without_internet").

**Hints**
- The altair example gallery includes a good choropleth example: https://altair-viz.github.io/gallery/choropleth.html
- See [altair documentation](https://altair-viz.github.io/user_guide/customization.html#color-schemes) on changing the color scheme and the [Vega documentation](https://vega.github.io/vega/docs/schemes/#reference) for the names of the allowed color schemes in altair
- You'll want to specify the projection type as "albersUsa" 

In [22]:
import altair as alt

## Leaflet/Folium plugins

One of leaflet's strengths: a rich set of open-source plugins

https://leafletjs.com/plugins.html

Many of these are available in Folium!

## Example: Heatmap

In [24]:
from folium.plugins import HeatMap

In [84]:
HeatMap?

Init signature:
HeatMap(
    data,
    name=None,
    min_opacity=0.5,
    max_zoom=18,
    radius=25,
    blur=15,
    gradient=None,
    overlay=True,
    control=True,
    show=True,
    **kwargs,
)
Docstring:     
Create a Heatmap layer

Parameters
----------
data : list of points of the form [lat, lng] or [lat, lng, weight]
    The points you want to plot.
    You can also provide a numpy.array of shape (n,2) or (n,3).
name : string, default None
    The name of the Layer, as it will appear in LayerControls.
min_opacity  : default 1.
    The minimum opacity the heat will start at.
max_zoom : default 18
    Zoom level where the points reach maximum intensity (as intensity
    scales with zoom), equals maxZoom of the map by default
radius : int, default 25
    Radius of each "point" of the heatmap
blur : int, default 15
    Amount of blur
gradient : dict, default None
    Color gradient config. e.g. {0.4: 'blue', 0.65: 'lime', 1: 'red'}
overlay : bool, default True
    Adds the laye

## Example: A heatmap of new construction permits in Philadelphia in the last 30 days

New construction in Philadelphia requires a building permit, which we can pull from Open Data Philly.

- Data available from OpenDataPhilly: https://www.opendataphilly.org/dataset/licenses-and-inspections-building-permits
- Query the database API directly to get the GeoJSON
- We can use the `carto2gpd` package to get the data
- API documentation: https://cityofphiladelphia.github.io/carto-api-explorer/#permits

### Step 1: Download the data from CARTO

**The "current_date" variable in SQL databases**

You can use the pre-defined "current_date" variable to get the current date. For example, to get the permits from the past 30 days, we could do:

``SELECT * FROM permits WHERE permitissuedate >= current_date - 30``


**Selecting only new construction permits**

To select new construction permits, you can use the "permitdescription" field. There are two relevant categories:

- "RESIDENTIAL BUILDING PERMIT"
- "COMMERCIAL BUILDING PERMIT"

We can use the SQL `IN` command ([documentation](https://www.w3schools.com/sql/sql_in.asp)) to easily select rows that have these categories.

In [25]:
import carto2gpd

In [26]:
# API URL
url = "https://phl.carto.com/api/v2/sql"

# Table name on CARTO
table_name = "permits"

# The where clause, with two parts
DAYS = 30
where = f"permitissuedate >= current_date - {DAYS}"
where += " and permitdescription IN ('RESIDENTIAL BUILDING PERMIT', 'COMMERCIAL BUILDING PERMIT')"

where

"permitissuedate >= current_date - 30 and permitdescription IN ('RESIDENTIAL BUILDING PERMIT', 'COMMERCIAL BUILDING PERMIT')"

In [27]:
# Run the query
permits = carto2gpd.get(url, table_name, where=where)

In [28]:
len(permits)

478

In [29]:
permits.head()

,geometry,cartodb_id,objectid,permitnumber,addressobjectid,parcel_id_num,permittype,permitdescription,commercialorresidential,typeofwork,...,unit_type,unit_num,zip,censustract,council_district,opa_owner,systemofrecord,geocode_x,geocode_y,posse_jobid
0,POINT (-75.16551 40.04532),4676,4898,CP-2022-002038,15732535,210522,BUILDING,COMMERCIAL BUILDING PERMIT,COMMERCIAL,ADDITION AND/OR ALTERATION,...,None,None,19144-1206,248,8,GAROFALO HELEN V,ECLIPSE,2.691986e+06,269953.280616,465300244
1,POINT (-75.00539 40.03600),4700,4904,CP-2022-003337,156760706,440714,BUILDING,COMMERCIAL BUILDING PERMIT,COMMERCIAL,NEW CONSTRUCTION,...,None,None,19136-1604,9891,6,PHILA MUNICIPAL AUTH,ECLIPSE,2.736901e+06,267914.438827,486707349
2,POINT (-75.09335 40.05158),4724,4910,CP-2022-004987,184460163,352755,BUILDING,COMMERCIAL BUILDING PERMIT,COMMERCIAL,ADDITION AND/OR ALTERATION,...,None,None,19111-5246,306,9,GRACE YANG INC,ECLIPSE,2.712111e+06,272832.652136,514998496
3,POINT (-75.16733 40.06235),5439,5684,RP-2022-005876,265435068,331238,RESIDENTIAL BUILDING,RESIDENTIAL BUILDING PERMIT,RESIDENTIAL,ADDITION AND/OR ALTERATION,...,None,None,19150-3300,264,9,CMPJ LLC,ECLIPSE,2.691293e+06,276139.761238,480078356
4,POINT (-75.22938 39.93547),5463,5687,RP-2022-007783,132249898,366914,RESIDENTIAL BUILDING,RESIDENTIAL BUILDING PERMIT,RESIDENTIAL,ADDITION AND/OR ALTERATION,...,None,None,19143-5516,65,2,DH1 FUND LP,ECLIPSE,2.675252e+06,229438.672154,497420889


### Step 2: Remove missing geometries

Some permits don't have locations — use the `.geometry.notnull()` function to trim the data frame to those incidents with valid geometries.

In [30]:
permits = permits.loc[permits.geometry.notnull()].copy()

### Step 3: Extract out the lat/lng coordinates

**Note:** We need an array of (latitude, longitude) pairs. Be careful about the order!

In [31]:
# Extract the lat and longitude from the geometery column
permits['lat'] = permits.geometry.y
permits['lng'] = permits.geometry.x

In [32]:
permits.head()

,geometry,cartodb_id,objectid,permitnumber,addressobjectid,parcel_id_num,permittype,permitdescription,commercialorresidential,typeofwork,...,zip,censustract,council_district,opa_owner,systemofrecord,geocode_x,geocode_y,posse_jobid,lat,lng
0,POINT (-75.16551 40.04532),4676,4898,CP-2022-002038,15732535,210522,BUILDING,COMMERCIAL BUILDING PERMIT,COMMERCIAL,ADDITION AND/OR ALTERATION,...,19144-1206,248,8,GAROFALO HELEN V,ECLIPSE,2.691986e+06,269953.280616,465300244,40.045317,-75.165507
1,POINT (-75.00539 40.03600),4700,4904,CP-2022-003337,156760706,440714,BUILDING,COMMERCIAL BUILDING PERMIT,COMMERCIAL,NEW CONSTRUCTION,...,19136-1604,9891,6,PHILA MUNICIPAL AUTH,ECLIPSE,2.736901e+06,267914.438827,486707349,40.036004,-75.005385
2,POINT (-75.09335 40.05158),4724,4910,CP-2022-004987,184460163,352755,BUILDING,COMMERCIAL BUILDING PERMIT,COMMERCIAL,ADDITION AND/OR ALTERATION,...,19111-5246,306,9,GRACE YANG INC,ECLIPSE,2.712111e+06,272832.652136,514998496,40.051579,-75.093348
3,POINT (-75.16733 40.06235),5439,5684,RP-2022-005876,265435068,331238,RESIDENTIAL BUILDING,RESIDENTIAL BUILDING PERMIT,RESIDENTIAL,ADDITION AND/OR ALTERATION,...,19150-3300,264,9,CMPJ LLC,ECLIPSE,2.691293e+06,276139.761238,480078356,40.062348,-75.167333
4,POINT (-75.22938 39.93547),5463,5687,RP-2022-007783,132249898,366914,RESIDENTIAL BUILDING,RESIDENTIAL BUILDING PERMIT,RESIDENTIAL,ADDITION AND/OR ALTERATION,...,19143-5516,65,2,DH1 FUND LP,ECLIPSE,2.675252e+06,229438.672154,497420889,39.935474,-75.229383


In [33]:
# Split out the residential and commercial
residential = permits.query("permitdescription == 'RESIDENTIAL BUILDING PERMIT'")
commercial = permits.query("permitdescription == 'COMMERCIAL BUILDING PERMIT'")

In [34]:
# Make a NumPy array (use the "values" attribute)
residential_coords = residential[['lat', 'lng']].values
commercial_coords = commercial[['lat', 'lng']].values

In [35]:
commercial_coords[:5]

array([[ 40.045317, -75.165507],
       [ 40.036004, -75.005385],
       [ 40.051579, -75.093348],
       [ 39.969465, -75.2167  ],
       [ 39.935832, -75.155138]])

### Step 4: Make a Folium map and add a HeatMap

The HeatMap takes the list of coordinates: **the first column is latitude and the second column longitude**

### Commercial building permits

In [36]:
# Initialize map
m = folium.Map(
    location=[39.99, -75.13],
    tiles='Cartodb Positron',
    zoom_start=12
)


# Add heat map coordinates
HeatMap(commercial_coords, radius=15).add_to(m)

m

### Residential building permits

In [37]:
# Initialize map
m = folium.Map(
    location=[39.99, -75.13],
    tiles='Cartodb Positron',
    zoom_start=12
)


# Add heat map
HeatMap(residential_coords, radius=15).add_to(m)

m

### Takeaways

Commercial construction concentrated in the greater Center City area while residential construction is primarily outside of Center City...

## That's it for interactive maps w/ Folium...


Now on to clustering...

## Clustering in Python

- Both spatial and non-spatial datasets
- Two new techniques: 
    - Non-spatial: **K-means**
    - Spatial: **DBSCAN**
- Two labs/exercises this week:
    1. Grouping Philadelphia neighborhoods by AirBnb listings
    1. Identifying clusters in taxi rides in NYC

## "Machine learning"

<img src="imgs/machine_learning.jpg" width=700>

- The computer **learns** patterns and properties of an input data set without the user specifying them beforehand
- Can be both **supervised** and **unsupervised**

### Supervised
- Example: classification
- Given a training set of **labeled** data, learn to assign labels to new data

### Unsupervised
- Example: clustering
- Identify structure / clusters in data without any prior knowledge

## Machine learning in Python: scikit-learn

- State-of-the-art machine learning in Python
- Easy to use, lots of functionality

## Clustering is just one (of many) features
<img src="imgs/scikit_learn.png" width=700>

https://scikit-learn.org/stable/
    
**Note:** We will focus on clustering algorithms today and discuss a few other machine learning techniques in the next two weeks. If there is a specific scikit-learn use case we won't cover, I'm open to ideas for incorporating it as part of the final project. 

## Part 1: Non-spatial clustering

**The goal**

Partition a dataset into groups that have a similar set of attributes, **or features**, within the group and a dissimilar set of features between groups.

Minimize the *intra-cluster* variance and maximize the *inter-cluster* variance of features.

## Some intuition
<img src="imgs/clustering.png" width=700>

## K-Means clustering

- Simple but robust clustering algorithm
- Widely used
- **Important:** user must specify the number of clusters
- **Cannot** be used to find density-based clusters

## This is just one of several clustering methods

https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods

## A good introduction

[Andrew Ng's Coursera lecture](https://www.youtube.com/watch?v=hDmNF9JG3lo&feature=youtu.be)

## How does it work?

**Minimizes the intra-cluster variance**: minimizes the sum of the squared distances between all points in a cluster and the cluster centroid

## K-means in action

<img src="imgs/K-means_convergence.gif" width=500>

## Example: Clustering countries by health and income

- Health expectancy in years vs. GDP per capita and population for 187 countries (as of 2015)
- Data from [Gapminder](https://www.gapminder.org/data/)

In [38]:
import altair as alt
from vega_datasets import data as vega_data

Read the data from a URL:

In [39]:
gapminder = pd.read_csv(vega_data.gapminder_health_income.url)
gapminder.head()

,country,income,health,population
0,Afghanistan,1925,57.63,32526562
1,Albania,10620,76.00,2896679
2,Algeria,13434,76.50,39666519
3,Andorra,46577,84.10,70473
4,Angola,7615,61.00,25021974


## Plot it with altair

In [40]:
(
    alt.Chart(gapminder)
    .mark_circle()
    .encode(
        alt.X("income:Q", scale=alt.Scale(type="log")),
        alt.Y("health:Q", scale=alt.Scale(zero=False)),
        size="population:Q",
        tooltip=list(gapminder.columns),
    )
    .properties(width=800, height=600)
    .interactive()
)

alt.Chart(...)

## K-Means with scikit-learn

In [41]:
from sklearn.cluster import KMeans

Let's start with 5 clusters

In [42]:
KMeans?

Init signature:
KMeans(
    n_clusters=8,
    *,
    init='k-means++',
    n_init=10,
    max_iter=300,
    tol=0.0001,
    verbose=0,
    random_state=None,
    copy_x=True,
    algorithm='lloyd',
)
Docstring:     
K-Means clustering.

Read more in the :ref:`User Guide <k_means>`.

Parameters
----------

n_clusters : int, default=8
    The number of clusters to form as well as the number of
    centroids to generate.

init : {'k-means++', 'random'}, callable or array-like of shape             (n_clusters, n_features), default='k-means++'
    Method for initialization:

    'k-means++' : selects initial cluster centroids using sampling based on
    an empirical probability distribution of the points' contribution to the
    overall inertia. This technique speeds up convergence, and is
    theoretically proven to be :math:`\mathcal{O}(\log k)`-optimal.
    See the description of `n_init` for more details.

    'random': choose `n_clusters` observations (rows) at random from data
    for

In [43]:
kmeans = KMeans(n_clusters=5)

Lot's of optional parameters, but `n_clusters` is the most important:

## Let's fit just income first

Use the `fit()` function

In [44]:
kmeans.fit(gapminder[['income']]);

## Extract the cluster labels

Use the `labels_` attribute

In [45]:
gapminder['label'] = kmeans.labels_

## How big are our clusters?

In [46]:
gapminder.groupby('label').size()

label
0    106
1     24
2      6
3     50
4      1
dtype: int64

## Plot it again, coloring by our labels

In [47]:
(
    alt.Chart(gapminder)
    .mark_circle()
    .encode(
        alt.X("income:Q", scale=alt.Scale(type="log")),
        alt.Y("health:Q", scale=alt.Scale(zero=False)),
        size="population:Q",
        color=alt.Color("label:N", scale=alt.Scale(scheme="dark2")),
        tooltip=list(gapminder.columns),
    )
    .properties(width=800, height=600)
    .interactive()
)

alt.Chart(...)

### Calculate average income by group

In [48]:
gapminder.groupby("label")['income'].mean().sort_values()

label
0      5279.830189
3     21040.820000
1     42835.500000
2     74966.166667
4    132877.000000
Name: income, dtype: float64

Data is nicely partitioned into income levels

## How about health, income, and population?

In [49]:
# Fit all three columns
kmeans.fit(gapminder[['income', 'health', 'population']])

# Extract the labels
gapminder['label'] = kmeans.labels_

## Plot the new labels

In [50]:
(
    alt.Chart(gapminder)
    .mark_circle()
    .encode(
        alt.X("income:Q", scale=alt.Scale(type="log")),
        alt.Y("health:Q", scale=alt.Scale(zero=False)),
        size="population:Q",
        color=alt.Color("label:N", scale=alt.Scale(scheme="dark2")),
        tooltip=list(gapminder.columns),
    )
    .properties(width=800, height=600)
    .interactive()
)

alt.Chart(...)

### It....didn't work that well

What's wrong?

**K-means is distance-based, but our features have wildly different distance scales**

## scikit-learn to the rescue: pre-processing

* Scikit-learn has a utility to normalize features with an average of zero and a variance of 1
* Use the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) class

In [51]:
from sklearn.preprocessing import MinMaxScaler

In [52]:
from sklearn.preprocessing import StandardScaler

In [53]:
scaler = StandardScaler()

## Use the `fit_transform()` function to scale your features

In [54]:
gapminder_scaled = scaler.fit_transform(gapminder[['income', 'health', 'population']])

**Important**: The `fit_transform()` function converts the DataFrame to a numpy array:

In [55]:
# fit_transform() converts the data into a numpy array
gapminder_scaled[:5]

array([[-0.79481258, -1.8171424 , -0.04592039],
       [-0.34333373,  0.55986273, -0.25325837],
       [-0.1972197 ,  0.62456075,  0.00404216],
       [ 1.52369617,  1.6079706 , -0.27303503],
       [-0.49936524, -1.38107777, -0.09843447]])

In [56]:
# mean of zero
gapminder_scaled.mean(axis=0)

array([ 8.07434927e-17, -1.70511258e-15, -1.89984689e-17])

In [57]:
# variance of one
gapminder_scaled.std(axis=0)

array([1., 1., 1.])

## Now fit the scaled features

In [58]:
# Perform the fit
kmeans.fit(gapminder_scaled)

# Extract the labels
gapminder['label'] = kmeans.labels_

In [59]:
(
    alt.Chart(gapminder)
    .mark_circle()
    .encode(
        alt.X("income:Q", scale=alt.Scale(type="log")),
        alt.Y("health:Q", scale=alt.Scale(zero=False)),
        size="population:Q",
        color=alt.Color("label:N", scale=alt.Scale(scheme="dark2")),
        tooltip=list(gapminder.columns),
    )
    .properties(width=800, height=600)
    .interactive()
)

alt.Chart(...)

In [60]:
# Number of countries per cluster
gapminder.groupby("label").size()

label
0    86
1     2
2    32
3    61
4     6
dtype: int64

In [61]:
# Average population per cluster
gapminder.groupby("label")['population'].mean().sort_values() / 1e6

label
4       2.988746
3      21.441296
0      26.239937
2      32.501032
1    1343.549735
Name: population, dtype: float64

In [62]:
# Average life expectancy per cluster
gapminder.groupby("label")['health'].mean().sort_values()

label
3    62.232951
1    71.850000
0    74.313837
2    80.806250
4    81.033333
Name: health, dtype: float64

In [63]:
# Average income per cluster
gapminder.groupby("label")['income'].mean().sort_values() / 1e3

label
3     4.150623
1     9.618500
0    13.229907
2    40.322094
4    86.987500
Name: income, dtype: float64

In [64]:
gapminder.loc[gapminder['label']==4]

,country,income,health,population,label
24,Brunei,73003,78.7,423188,4
88,Kuwait,82633,80.7,3892115,4
97,Luxembourg,88314,81.1,567110,4
124,Norway,64304,81.6,5210967,4
134,Qatar,132877,82.0,2235355,4
145,Singapore,80794,82.1,5603740,4


In [65]:
kmeans.inertia_

80.84493050696736

In [66]:
gapminder.loc[gapminder['label']==2]

,country,income,health,population,label
3,Andorra,46577,84.1,70473,2
8,Australia,44056,81.8,23968973,2
9,Austria,44401,81.0,8544586,2
12,Bahrain,44138,79.2,1377237,2
16,Belgium,41240,80.4,11299192,2
30,Canada,43294,81.7,35939927,2
44,Cyprus,29797,82.6,1165300,2
45,Czech Republic,29437,78.6,10543186,2
46,Denmark,43495,80.1,5669081,2
58,Finland,38923,80.8,5503457,2


## Exercise: Clustering neighborhoods by Airbnb stats

I've extracted neighborhood Airbnb statistics for Philadelphia neighborhoods from [Tom Slee's website](http://tomslee.net/airbnb-data).

The data includes average price per person, overall satisfaction, and number of listings.

### Step 1: Load the data with pandas

The data is available in CSV format ("philly_airbnb_by_neighborhoods.csv") in the "data/" folder of the repository.

### Step 2: Perform the K-Means fit

- Use our three features: `price_per_person`, `overall_satisfaction`, `N`
- I used 5 clusters, but you are welcome to experiment with different values! We will discuss what the *optimal* number is after we go through the solutions!
- Scaling the features is recommended, but if the scales aren't too different, so probably isn't necessary in this case

### Step 3: Calculate average features per cluster

To gain some insight into our clusters, after calculating the K-Means labels:
- group by the `label` column
- calculate the `mean()` of each of our features
- calculate the number of neighborhoods per cluster

### Step 4: Plot a choropleth, coloring neighborhoods by their cluster label

- Part 1: Load the Philadelphia neighborhoods available in the data directory:  
    - `./data/philly_neighborhoods.geojson`
- Part 2: Merge the Airbnb data (with labels) and the neighborhood polygons
- Part 3: Use geopandas to plot the neighborhoods
    - The `categorical=True` and `legend=True` keywords will be useful here

### Step 5: Plot an interactive map

Use altair to plot the clustering results with a tooltip for neighborhood name and tooltip.

**Hint:** See [week 3B's lecture](https://musa-550-fall-2022.github.io/slides/lecture-3B.html) on interactive choropleth's with altair

## Based on these results, where would you want to stay?



## More clustering on Wednesday!